In [18]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import os 
import torch.nn.functional as F

import importlib
import data_analysis
importlib.reload(data_analysis)

import utils
importlib.reload(utils)

<module 'utils' from '/home/jlimonet/Documents/Stage_M2/code/utils.py'>

In [2]:
from utils import get_freer_gpu

device = get_freer_gpu() if torch.cuda.is_available() else "cpu"

Selected GPU 0 with 20675 MB free memory 


In [22]:
class SimuDataset(Dataset):

    def __init__(self, simu, rgb = False, transform = None):
        X = simu.X
        if rgb : 
            X = simu.X_rgb
        self.x = torch.from_numpy(X).to(device)
        self.n_snapshots = X.shape[1]
        self.transform = transform
        self.rgb = rgb
        
    def __getitem__(self, index):
        sample = self.x[:,index]
        
        if self.transform:
            sample = self.transform(sample)
        if self.rgb:
            sample = self.x[:,index, :, :]
        return sample
    
    def __len__(self):
        return self.n_snapshots
    
    def plot_field(self, t, save=False, directory=None):
        fig, ax = plt.subplots(figsize=(15, 5))
        vmin = self.T[t, :, :].min()
        vmax = self.T[t, :, :].max()
        abs_max = max(abs(vmin), abs(vmax))
        ax.streamplot(
            self.x.T,
            self.z.T,
            self.u[t, :, :].T,
            self.w[t, :, :].T,
            color="k",
            arrowsize=0.7,
            linewidth=1,
        )
        levels = np.linspace(vmin, vmax, 20)
        cf0 = ax.contourf(
            self.x,
            self.z,
            self.T[t, :, :],
            levels=levels,
            cmap=cm.Spectral.reversed(),
            norm=matplotlib.colors.Normalize(vmin=vmin, vmax=vmax),
        )
        cbar = plt.colorbar(cf0, ax=ax, shrink=0.35, aspect=6, ticks= [-abs_max, 0, abs_max])
        cbar.ax.set_aspect("auto")
        ax.set_title(f"Temperature and velocity field at t = {t}")
        ax.set_aspect("equal")
        ax.set_ylim(0, 1)
        ax.set_xlim(-4, 4)


        if save:
            plt.savefig(directory, dpi=300)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()

class LinearAutoencoder(nn.Module):
    def __init__(self, n_channels, N):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_channels*N, 128*16), 
            nn.ReLU(), 
            nn.Linear(128*16, 64*16), 
            nn.ReLU(), 
            nn.Linear(64*16, 12*16))
        
        self.decoder = nn.Sequential(
            nn.Linear(12*16, 64*16), 
            nn.ReLU(), 
            nn.Linear(64*16, 128*16), 
            nn.ReLU(), 
            nn.Linear(128*16, n_channels*N),
            nn.Tanh())
        
    def forward(self, x):
        # print(f'size of x {x.shape}')
        x = x.float()
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class SimpleLinearAutoencoder(nn.Module):
    def __init__(self, N, device):
        super().__init__()
        self.encoder = nn.Linear(N, 128).to(device)
        
        self.decoder = nn.Sequential(
            nn.Linear(128, N), 
            nn.Tanh()).to(device)
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [19]:
from data_analysis import Simulation
from data_analysis import compatible_path
current_directory = compatible_path('../')

simulation = Simulation(current_directory, normalize = True)
time, x, z, u, w, T, umean, wmean, Tmean = map(lambda x: torch.tensor(x).to(device), simulation.import_data())
simulation.image_rgb()

h, l = np.shape(x)
m = len(time)
print(m)
N = h*l*3

6249


In [9]:
batch_size = 4

dataset = SimuDataset(simulation)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)

model = SimpleLinearAutoencoder(N, device = device)
model.to(device) 
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

6249
tensor(-0.6515, device='cuda:0', dtype=torch.float64) tensor(0.5522, device='cuda:0', dtype=torch.float64) torch.Size([4, 12393])


In [10]:
num_epoch = 9
output = []
for epoch in range(num_epoch):
    for map in dataloader:
        map = map.float().to(device)
        reconstructed = model(map)
        loss = criterion(reconstructed, map)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss:{loss.item() : .4f}")
    output.append((epoch, map, reconstructed))

Epoch: 1, Loss: 0.0236
Epoch: 2, Loss: 0.0174
Epoch: 3, Loss: 0.0219
Epoch: 4, Loss: 0.0353
Epoch: 5, Loss: 0.0174
Epoch: 6, Loss: 0.0239
Epoch: 7, Loss: 0.0346
Epoch: 8, Loss: 0.0282
Epoch: 9, Loss: 0.0409


In [8]:
print(model.encoder)

Linear(in_features=12393, out_features=128, bias=True)


In [16]:
3*81*51

12393

In [27]:
8 * 27 * 17

3672

In [29]:
24/3

8.0

In [115]:
import torch.nn.functional as F
size1 = (20, 12)
size2 = (10, 6)
size3 = (3, 2)

class ConvolutionalAutoencoder(nn.Module):

    def __init__(self, device, n_channels = 64, kernel_size = 3, stride = 1, padding = 1):
        super().__init__()
        #shape : B * 3 * 81 * 51
        self.encoder = nn.Sequential(
            nn.Conv2d(3, n_channels//2, kernel_size, stride = stride, padding = padding), 
            nn.ReLU(), 
            nn.Upsample(size=size1, mode='bilinear', align_corners=False), 
            nn.Conv2d(n_channels//2, n_channels, kernel_size, stride = stride, padding = padding),
            nn.ReLU(),
            nn.Upsample(size=size2, mode='bilinear', align_corners=False), 
            nn.Conv2d(n_channels, n_channels, kernel_size, stride = stride, padding = padding), 
            nn.Upsample(size=size3, mode='bilinear', align_corners=False)).to(device)
        
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(3, n_channels//2, kernel_size, stride = stride, padding = padding), 
            nn.ReLU(), 
            nn.Upsample(size=size2, mode='bilinear', align_corners=False), 
            nn.ConvTranspose2d(n_channels//2, n_channels, kernel_size, stride = stride, padding = padding),
            nn.ReLU(),
            nn.Upsample(size=size1, mode='bilinear', align_corners=False), 
            nn.ConvTranspose2d(n_channels, n_channels, kernel_size, stride = stride, padding = padding), 
            nn.Upsample(size=(81, 51), mode='bilinear', align_corners=False)).to(device)
        
        
    def forward(self, x):
        print(f'size of x {x.shape}')
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [117]:
batch_size = 4

dataset = SimuDataset(simulation, rgb = True)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)

model = ConvolutionalAutoencoder(device = device)
model.to(device) 
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

tensor(-0.6562, device='cuda:0', dtype=torch.float64) tensor(0.5647, device='cuda:0', dtype=torch.float64) torch.Size([4, 3, 81, 51])


In [118]:
num_epoch = 9
output = []
for epoch in range(num_epoch):
    for map in dataloader:
        map = map.float().to(device)
        reconstructed = model(map)
        loss = criterion(reconstructed, map)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss:{loss.item() : .4f}")
    output.append((epoch, map, reconstructed))

size of x torch.Size([4, 3, 81, 51])


RuntimeError: Given transposed=1, weight of size [3, 32, 3, 3], expected input[4, 64, 3, 2] to have 3 channels, but got 64 channels instead

In [21]:
print(np.shape(simulation.X_rgb))

(3, 6249, 81, 51)


In [23]:
batch_size = 4

dataset = SimuDataset(simulation, rgb = True)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)

model = ConvolutionalAutoencoder(device = device)
model.to(device) 
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

tensor(-0.6562, device='cuda:0', dtype=torch.float64) tensor(0.5647, device='cuda:0', dtype=torch.float64) torch.Size([4, 3, 81, 51])


In [59]:
import torch.nn.functional as F
class ConvolutionalAutoencoder(nn.Module):

    def __init__(self, device, kernel_size = 3, stride = 1, padding = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, stride = stride).to(device) 
        self.conv2 = nn.Conv2d(6, 12, 5).to(device) 
        self.conv3 = nn.Conv2d(12, 28, 5).to(device) 

    def forward(self, x):
        x = self.conv1(x)
        print('Shape after conv1:', x.shape)
        x = F.relu(x)
        x = self.conv2(x)
        print('Shape after conv2:', x.shape)
        x = F.relu(x)
        x = self.conv3(x)
        print('Shape after conv3:', x.shape)
        return x

In [107]:
81*51*3

12393

In [112]:
64*20*12

15360

In [114]:
import torch.nn.functional as F

kernel_size = 3
stride = 1
padding = 1

size1 = (20, 12)
size2 = (10, 6)
size3 = (3, 2)
n_channels = 64

conv1 = nn.Conv2d(3, n_channels//2, kernel_size, stride = 1, padding = 1).to(device) 
conv2 = nn.Conv2d(n_channels//2, n_channels, kernel_size, stride = 1, padding = 1).to(device) 
conv3 = nn.Conv2d(n_channels, n_channels, kernel_size, stride = 1, padding = 1).to(device)
conv_inv = nn.ConvTranspose2d(n_channels, n_channels, kernel_size, stride = 1, padding = 1).to(device)
conv_inv2 = nn.ConvTranspose2d(n_channels, n_channels, kernel_size, stride = 1, padding = 1).to(device)
conv_inv3 = nn.ConvTranspose2d(n_channels, n_channels, kernel_size, stride = 1, padding = 1).to(device)


x = conv1(data.float()).to(device)

x = F.relu(x)
x = F.interpolate(x, size = size1)
print('Shape after conv1:', x.shape)
x = conv2(x)

x = F.relu(x)
x = F.interpolate(x, size = size2)
print('Shape after conv2:', x.shape)
x = conv3(x)
x = F.interpolate(x, size = size3)
print('Shape after conv3:', x.shape)

x = conv_inv(x)
x = F.interpolate(x, size = size2)
print('Shape after conv_inv1:', x.shape)
x = F.relu(x)
x = conv_inv2(x)
x = F.interpolate(x, size = size1)
print('Shape after conv_inv2:', x.shape)
x = F.relu(x)
x = conv_inv3(x)
x = F.interpolate(x, size = (81,51))
print('Shape after conv_inv3:', x.shape)

Shape after conv1: torch.Size([4, 32, 20, 12])
Shape after conv2: torch.Size([4, 64, 10, 6])
Shape after conv3: torch.Size([4, 64, 3, 2])
Shape after conv_inv1: torch.Size([4, 64, 10, 6])
Shape after conv_inv2: torch.Size([4, 64, 20, 12])
Shape after conv_inv3: torch.Size([4, 64, 81, 51])


In [72]:
batch_size = 4
n_channels = 3


dataset = SimuDataset(simulation, rgb = True)
first = dataset[0]
dataloader = DataLoader(dataset = dataset, batch_size = batch_size)

dataiter = iter(dataloader)
data = next(dataiter)
print(torch.min(data), torch.max(data), data.shape)
model = ConvolutionalAutoencoder(batch_size, n_channels, N)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

tensor(-0.7316, dtype=torch.float64) tensor(0.7301, dtype=torch.float64) torch.Size([4, 3, 81, 51])


In [49]:
print(np.shape(simulation.X_rgb))
print(np.shape(simulation.u))
print(4*3*81*51)
print(4*6*40*25)

(3, 6249, 81, 51)
(6249, 81, 51)
49572
24000


In [63]:
num_epoch = 1
output = []
for epoch in range(num_epoch):
    for map in dataloader:
        map = map.float()
        print(f'map shape : {map.shape}')
        reconstructed = model(map)
        print(f'reconstructed shape : {reconstructed.shape}')
        loss = criterion(reconstructed, map)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss:{loss.item() : .4f}")
    output.append((epoch, map, reconstructed))

map shape : torch.Size([4, 3, 81, 51])
Shape after conv1: torch.Size([4, 6, 40, 25])
Shape after conv2: torch.Size([4, 12, 19, 12])
Shape after conv3: torch.Size([4, 28, 15, 8])
reconstructed shape : torch.Size([4, 28, 15, 8])


/home/julielimonet/Documents/Stage_M2/code/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 3, 81, 51])) that is different to the input size (torch.Size([4, 28, 15, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (51) at non-singleton dimension 3